In [10]:
import efficientnet.keras as efn
import tensorflow as tf

from keras.applications import VGG16
from keras import layers, Model, Sequential
from keras.optimizers import RMSprop
from keras.callbacks import LearningRateScheduler

import numpy as np

In [11]:
!nvidia-smi

Tue Jun 23 05:23:46 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro P6000        On   | 00000000:00:05.0 Off |                  Off |
| 26%   25C    P0    59W / 250W |  23324MiB / 24449MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [12]:
from tensorflow.python.client import device_lib

In [13]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17270575579270919320, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13279518378790752503
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1311084198433950046
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 24199030375
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 18092526041124334047
 physical_device_desc: "device: 0, name: Quadro P6000, pci bus id: 0000:00:05.0, compute capability: 6.1"]

In [14]:
INPUT_SHAPE = (150, 150, 3)

In [15]:
NUM_CLASSES = 7

In [16]:
efn_base = efn.EfficientNetB0(
    weights='imagenet',
    include_top=False,
    classes=NUM_CLASSES,
    input_shape=INPUT_SHAPE
)

In [17]:
# efn_base.summary()

In [18]:
model = Sequential()

model.add(efn_base)
model.add(layers.GlobalMaxPooling2D())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

In [19]:
# model.summary()

In [20]:
for layer in efn_base.layers:
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

In [21]:
from h5imagegenerator import HDF5ImageGenerator

In [22]:
from albumentations import (
    Compose, ShiftScaleRotate, HorizontalFlip
)

In [23]:
aug = Compose([
    ShiftScaleRotate(
        shift_limit=0.0225,
        scale_limit=0.06,
        rotate_limit=30
    ),
])

In [24]:
train_gen = HDF5ImageGenerator(
    src= './train.h5',
    scaler=False,
    labels_encoding='hot',
    batch_size=128)

val_gen = HDF5ImageGenerator(
    src= './val.h5',
    scaler=False,
    labels_encoding='hot',
    batch_size=128)

In [25]:
def step_decay(epoch):
    factor    = 0.5
    drop_rate = 5

    return float(0.001 * (factor ** np.floor((1 + epoch) / drop_rate)))

In [26]:
rms = RMSprop(learning_rate=2e-5)

model.compile(
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    optimizer=rms
)

h = model.fit(
    train_gen,
    validation_data=val_gen,
    callbacks=[LearningRateScheduler(step_decay)],
    verbose=1,
    use_multiprocessing=True,
    workers=6,
    epochs=30
)


Epoch 1/30
112/112 [==============================] - 52s 466ms/step - loss: 3.0349 - accuracy: 0.3662 - val_loss: 1.7249 - val_accuracy: 0.5171
Epoch 2/30
112/112 [==============================] - 37s 330ms/step - loss: 2.0336 - accuracy: 0.4861 - val_loss: 1.1931 - val_accuracy: 0.5892
Epoch 3/30
112/112 [==============================] - 37s 329ms/step - loss: 1.6417 - accuracy: 0.5310 - val_loss: 1.4768 - val_accuracy: 0.5620
Epoch 4/30
112/112 [==============================] - 37s 332ms/step - loss: 1.4561 - accuracy: 0.5544 - val_loss: 1.2283 - val_accuracy: 0.6245
Epoch 5/30
112/112 [==============================] - 37s 332ms/step - loss: 1.2318 - accuracy: 0.5872 - val_loss: 1.1090 - val_accuracy: 0.6290
Epoch 6/30
112/112 [==============================] - 37s 332ms/step - loss: 1.1648 - accuracy: 0.6022 - val_loss: 0.7990 - val_accuracy: 0.6618
Epoch 7/30
112/112 [==============================] - 37s 333ms/step - loss: 1.1036 - accuracy: 0.6104 - val_loss: 1.0751 - val_a

Process ForkPoolWorker-275:
Process ForkPoolWorker-265:
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
Process ForkPoolWorker-274:
Process ForkPoolWorker-272:
Process ForkPoolWorker-270:
Process ForkPoolWorker-271:
Process ForkPoolWorker-268:
Process ForkPoolWorker-276:
Process ForkPoolWorker-266:
Process ForkPoolWorker-269:
Process ForkPoolWorker-267:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-273:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  Fil

KeyboardInterrupt: 